In [1]:
import requests

BASE_URL = 'http://127.0.0.1:8000/'

# Restaurants

First of all, we create a new restaurant, by "wrapping" our Django API inside a Python function.

In [2]:
def create_restaurant(name):
    url = BASE_URL + "restaurants/"
    data = {"name": name}
    response = requests.post(url, data=data)
    return response.json()

In [3]:
new_restaurant = create_restaurant("ToXAmato")
restaurant_id = new_restaurant['id']

In [4]:
restaurant_id

'ef98a8a6-6a13-47bf-8c10-f04298873250'

It is possible to have multiple restaurants with the same name; the unique uuid identifier will be different.

In [5]:
new_restaurant_2 = create_restaurant("ToXAmato")
restaurant_id_2 = new_restaurant_2['id']

In [6]:
restaurant_id_2

'17d2b8a7-3403-4226-be42-6b2ef8395997'

In [7]:
new_restaurant_2

{'id': '17d2b8a7-3403-4226-be42-6b2ef8395997', 'name': 'ToXAmato'}

We can also checks how many restaurants we have

In [9]:
restaurants = requests.get(BASE_URL + 'restaurants/')
restaurants.json()

[{'id': 'ef98a8a6-6a13-47bf-8c10-f04298873250', 'name': 'ToXAmato'},
 {'id': '17d2b8a7-3403-4226-be42-6b2ef8395997', 'name': 'ToXAmato'}]

The names can be modified

In [10]:
def modify_restaurant(restaurant_id, new_name):
    url = BASE_URL + f"restaurants/{restaurant_id}/"
    data = {"name": new_name}
    response = requests.put(url, data=data)
    return response.json()

In [11]:
new_restaurant = modify_restaurant(restaurant_id, "Tomato")
new_restaurant

{'id': 'ef98a8a6-6a13-47bf-8c10-f04298873250', 'name': 'Tomato'}

In [12]:
restaurant_id

'ef98a8a6-6a13-47bf-8c10-f04298873250'

Finally, restaurants can be deleted. We'll keep _Tomato's_, and remove _ToXAmato's_. The API returns _204_, as the operation was successfully completed.

In [13]:
def delete_restaurant(restaurant_id):
    url = BASE_URL + f"restaurants/{restaurant_id}/"
    response = requests.delete(url)
    return response.status_code

In [14]:
delete_restaurant(restaurant_id_2)


204

Before exploring recipes and ingredients, let's create some other restaurants. For practical reasons, we'll store their ids on a dictionary

In [15]:
create_restaurant("Cheesy")
create_restaurant("SushiKing")
create_restaurant("PizzaPazza")

restaurants = requests.get(BASE_URL + 'restaurants/')
restaurants.json()

[{'id': 'ef98a8a6-6a13-47bf-8c10-f04298873250', 'name': 'Tomato'},
 {'id': '45d3ae47-9445-45b2-9838-330dcc568422', 'name': 'Cheesy'},
 {'id': '2a087183-2e7e-4a9d-a982-21d837d01793', 'name': 'SushiKing'},
 {'id': '705046ab-df7c-4845-8017-24953bf51c68', 'name': 'PizzaPazza'}]

In [16]:
restaurant_ids = {e['name']: e['id'] for e in restaurants.json()}
restaurant_ids

{'Tomato': 'ef98a8a6-6a13-47bf-8c10-f04298873250',
 'Cheesy': '45d3ae47-9445-45b2-9838-330dcc568422',
 'SushiKing': '2a087183-2e7e-4a9d-a982-21d837d01793',
 'PizzaPazza': '705046ab-df7c-4845-8017-24953bf51c68'}

# Recipes

A _recipe_ can be created following a similar rationale.  Each recipe is associated to a restaurant -- and we need to provide the restaurant's unique identifier. Multiple recipes with the same name can co-exist, even inside the same restaurant.

In [17]:
def create_recipe(restaurant_id, recipe_name):
    url = BASE_URL + "/api/recipes/"
    data = {"name": recipe_name, "restaurant": restaurant_id}
    response = requests.post(url, data=data)
    return response.json()

new_recipe = create_recipe(restaurant_ids['Tomato'], "Pizza Margherita")
new_recipe

{'id': '184866b8-2eb8-421b-9b09-1233f8a4c0f9',
 'name': 'Pizza Margherita',
 'restaurant': 'ef98a8a6-6a13-47bf-8c10-f04298873250'}

In [18]:
create_recipe(restaurant_ids['PizzaPazza'], "Quattro Formaggi")
create_recipe(restaurant_ids['PizzaPazza'], "Pizza Marinara")
create_recipe(restaurant_ids['Cheesy'], "The Exclusive")
create_recipe(restaurant_ids['SushiKing'], "Sushi")
create_recipe(restaurant_ids['SushiKing'], "Dorayaki")
create_recipe(restaurant_ids['Tomato'], "Amatriciana")


recipes = requests.get(BASE_URL + 'recipes/') #Created recipes, as before we then store then in a dict

recipe_ids = {e['name']: e['id'] for e in recipes.json()}
recipe_ids

{'Pizza Margherita': '184866b8-2eb8-421b-9b09-1233f8a4c0f9',
 'Quattro Formaggi': 'deb5f0fa-fbdd-415a-9af0-bdd4eba37bc0',
 'Pizza Marinara': '36e8100a-b77c-47ec-9f71-c0601abeaceb',
 'The Exclusive': '0e8d97ba-a7e3-4450-a784-7e6b4599e921',
 'Sushi': 'e0bc2526-1677-4866-ade9-73a84cbcff09',
 'Dorayaki': 'f74558ee-eed7-462c-a07a-858410bf74c0',
 'Amatriciana': 'd8436372-88e0-4dd9-aca0-40b30fa46e0e'}

What if our friends at Cheesy's decide to sell the rights of their "The Exclusive" pizza to PizzaPazza? We can modify the name of each recipe, as well as the linked restaurant.

In [19]:
def modify_recipe(recipe_id, new_name, new_restaurant):
    url = BASE_URL + f"api/recipes/{recipe_id}/"
    data = {"name": new_name, 'restaurant': new_restaurant}
    response = requests.put(url, data=data)
    return response.json()

modified_recipe = modify_recipe(recipe_ids['The Exclusive'], new_name="Exclusive (Now With A New Owner!)",
                               new_restaurant=restaurant_ids['PizzaPazza'])

modified_recipe

{'id': '0e8d97ba-a7e3-4450-a784-7e6b4599e921',
 'name': 'Exclusive (Now With A New Owner!)',
 'restaurant': '705046ab-df7c-4845-8017-24953bf51c68'}

Finally, recipes can be deleted too.

In [20]:
def delete_recipe(recipe_id):
    url = BASE_URL + f"api/recipes/{recipe_id}/"
    response = requests.delete(url)
    return response.status_code

delete_recipe(recipe_ids['Sushi'])


204

As you can see below, deleting "SushiKing" also remove the "Dorayaki" recipes (since we are checking again all the recipes, we take to opportunity to update our dict as well)

In [21]:
delete_restaurant(restaurant_ids['SushiKing']) 

recipes = requests.get(BASE_URL + 'recipes/') #Created recipes, as before we then store then in a dict
recipe_ids = {e['name']: e['id'] for e in recipes.json()}
recipes.json()


[{'id': '184866b8-2eb8-421b-9b09-1233f8a4c0f9',
  'name': 'Pizza Margherita',
  'restaurant': 'ef98a8a6-6a13-47bf-8c10-f04298873250'},
 {'id': 'deb5f0fa-fbdd-415a-9af0-bdd4eba37bc0',
  'name': 'Quattro Formaggi',
  'restaurant': '705046ab-df7c-4845-8017-24953bf51c68'},
 {'id': '36e8100a-b77c-47ec-9f71-c0601abeaceb',
  'name': 'Pizza Marinara',
  'restaurant': '705046ab-df7c-4845-8017-24953bf51c68'},
 {'id': 'd8436372-88e0-4dd9-aca0-40b30fa46e0e',
  'name': 'Amatriciana',
  'restaurant': 'ef98a8a6-6a13-47bf-8c10-f04298873250'},
 {'id': '0e8d97ba-a7e3-4450-a784-7e6b4599e921',
  'name': 'Exclusive (Now With A New Owner!)',
  'restaurant': '705046ab-df7c-4845-8017-24953bf51c68'}]

# Ingredients

Finally, the ingredients. Ingredients can be linked to multiple recipes, and can exist even if their associated recipes are removed (they cannot, however, be created _without_ an initial linked recipe). 

In [22]:
def create_ingredient(recipe_id, ingredient_name):
    url = BASE_URL + "api/ingredients/"
    data = {"name": ingredient_name, "recipes": [recipe_id]}
    response = requests.post(url, data=data)
    return response.json()

gorgonzola = create_ingredient(recipe_ids['Quattro Formaggi'], "Gorgonzola")
gorgonzola_id = gorgonzola['id']


Note that a new ingredient can be associated to multiple recipes

In [23]:
new_ingredient = create_ingredient([recipe_ids['Pizza Margherita'], recipe_ids['Amatriciana'], 
                                    recipe_ids['Pizza Marinara']], "Pomodoro")
new_ingredient

{'id': 'd5b16057-f82d-48b5-ad1a-e848c138ca76',
 'name': 'Pomodoro',
 'recipes': ['184866b8-2eb8-421b-9b09-1233f8a4c0f9',
  '36e8100a-b77c-47ec-9f71-c0601abeaceb',
  'd8436372-88e0-4dd9-aca0-40b30fa46e0e']}

We cannot have ingredients with the same name

In [24]:
new_ingredient = create_ingredient(recipe_ids['Pizza Margherita'], 
                                   ingredient_name="Gorgonzola")
new_ingredient

{'name': ['ingredient with this Name already exists.']}

We can change the name of an ingredient, or its recipes

In [25]:
def modify_ingredient(ingredient_id, new_name, recipes):
    url = BASE_URL + f"api/ingredients/{ingredient_id}/"
    data = {"name": new_name, 'recipes': recipes}
    response = requests.put(url, data=data)
    return response.json()

modified_ingredient = modify_ingredient(ingredient_id=gorgonzola['id'], 
                                        new_name="Formaggio Gorgonzola",
                                       recipes=recipe_ids['Quattro Formaggi'])


In [26]:
modified_ingredient

{'id': '59acd7a0-9726-45d4-81c6-f0a2b5c0493b',
 'name': 'Formaggio Gorgonzola',
 'recipes': ['deb5f0fa-fbdd-415a-9af0-bdd4eba37bc0']}

Ingredients can be deleted as well

In [27]:
def delete_ingredient(ingredient_id):
    url = BASE_URL + f"api/ingredients/{ingredient_id}/"
    response = requests.delete(url)
    return response.status_code

delete_ingredient(ingredient_id=gorgonzola_id)


204

Before going further, let's add some ingredients to our recipes and create the dictionary

In [28]:
create_ingredient(recipe_ids['Amatriciana'], 
                                   ingredient_name="Guanciale")
create_ingredient(recipe_ids['Amatriciana'], 
                                   ingredient_name="Pancetta")
create_ingredient(recipe_ids['Pizza Margherita'], 
                                   ingredient_name="Mozzarella")

ingredients = requests.get(BASE_URL + 'ingredients/')
ingredient_ids = {e['name']: e['id'] for e in ingredients.json()}
ingredient_ids

{'Pomodoro': 'd5b16057-f82d-48b5-ad1a-e848c138ca76',
 'Guanciale': '8d6aabda-b69c-4187-b6c3-6fdd340f6e95',
 'Pancetta': '4634145f-36e8-4918-b116-0880f31da0bc',
 'Mozzarella': '18a5dcc5-50ee-43f9-bc34-d763e6a5ef6a'}

# Exploring the DB

We want to find the restaurants serving a given recipe. Given the nature of the request, we assume multiple recipes with the same name, served by different restaurants. We already know that Tomato serves Pizza Margherita, so we create another (independent) "Pizza Margherita" served by Pizza Pazza.

In [29]:
create_recipe(restaurant_ids['PizzaPazza'], "Pizza Margherita")

{'id': 'ecc45894-dc91-4fe3-9da0-e6bee27dad85',
 'name': 'Pizza Margherita',
 'restaurant': '705046ab-df7c-4845-8017-24953bf51c68'}

In [30]:
recipe_name = "Pizza Margherita"  # Replace with the actual name of the recipe

response = requests.get(BASE_URL + f'restaurants/by_recipe/{recipe_name}/')
restaurants = response.json()
restaurants

[{'id': 'ef98a8a6-6a13-47bf-8c10-f04298873250', 'name': 'Tomato'},
 {'id': '705046ab-df7c-4845-8017-24953bf51c68', 'name': 'PizzaPazza'}]

The opposite. We want to find the recipes served by a specific restaurant. In this case, the restaurant ID will do.

In [31]:
restaurant_id = restaurant_ids['Tomato']

response = requests.get(BASE_URL + f'/restaurants/{restaurant_id}/recipes/')

recipes = response.json()
recipes

[{'id': '184866b8-2eb8-421b-9b09-1233f8a4c0f9',
  'name': 'Pizza Margherita',
  'restaurant': 'ef98a8a6-6a13-47bf-8c10-f04298873250'},
 {'id': 'd8436372-88e0-4dd9-aca0-40b30fa46e0e',
  'name': 'Amatriciana',
  'restaurant': 'ef98a8a6-6a13-47bf-8c10-f04298873250'}]

Given an ingredient, we want to find the recipes

In [32]:
ingredient_id = ingredient_ids['Pomodoro']
response = requests.get(BASE_URL + f'/ingredients/{ingredient_id}/recipes/')

# Print the list of recipes
recipes = response.json()
recipes

[{'id': 'd8436372-88e0-4dd9-aca0-40b30fa46e0e',
  'name': 'Amatriciana',
  'restaurant': 'ef98a8a6-6a13-47bf-8c10-f04298873250'},
 {'id': '184866b8-2eb8-421b-9b09-1233f8a4c0f9',
  'name': 'Pizza Margherita',
  'restaurant': 'ef98a8a6-6a13-47bf-8c10-f04298873250'},
 {'id': '36e8100a-b77c-47ec-9f71-c0601abeaceb',
  'name': 'Pizza Marinara',
  'restaurant': '705046ab-df7c-4845-8017-24953bf51c68'}]

The opposite. Given a recipe, the employed ingredients

In [33]:
recipe_id = recipe_ids['Amatriciana']

response = requests.get(BASE_URL + f"recipes/{recipe_id}/ingredients/")
response.json()


[{'id': 'd5b16057-f82d-48b5-ad1a-e848c138ca76',
  'name': 'Pomodoro',
  'recipes': ['d8436372-88e0-4dd9-aca0-40b30fa46e0e',
   '184866b8-2eb8-421b-9b09-1233f8a4c0f9',
   '36e8100a-b77c-47ec-9f71-c0601abeaceb']},
 {'id': '8d6aabda-b69c-4187-b6c3-6fdd340f6e95',
  'name': 'Guanciale',
  'recipes': ['d8436372-88e0-4dd9-aca0-40b30fa46e0e']},
 {'id': '4634145f-36e8-4918-b116-0880f31da0bc',
  'name': 'Pancetta',
  'recipes': ['d8436372-88e0-4dd9-aca0-40b30fa46e0e']}]

Given a restaurant, the employed ingredients

In [34]:
restaurant_id = restaurant_ids['Tomato']

response = requests.get(BASE_URL + f'/restaurants/{restaurant_id}/ingredients/')
response.json()


[{'id': '18a5dcc5-50ee-43f9-bc34-d763e6a5ef6a',
  'name': 'Mozzarella',
  'recipes': ['184866b8-2eb8-421b-9b09-1233f8a4c0f9']},
 {'id': '4634145f-36e8-4918-b116-0880f31da0bc',
  'name': 'Pancetta',
  'recipes': ['d8436372-88e0-4dd9-aca0-40b30fa46e0e']},
 {'id': '8d6aabda-b69c-4187-b6c3-6fdd340f6e95',
  'name': 'Guanciale',
  'recipes': ['d8436372-88e0-4dd9-aca0-40b30fa46e0e']},
 {'id': 'd5b16057-f82d-48b5-ad1a-e848c138ca76',
  'name': 'Pomodoro',
  'recipes': ['d8436372-88e0-4dd9-aca0-40b30fa46e0e',
   '184866b8-2eb8-421b-9b09-1233f8a4c0f9',
   '36e8100a-b77c-47ec-9f71-c0601abeaceb']}]